## Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models.vgg import VGG
from PIL import Image
import numpy as np
import os
from tqdm import tqdm

## Dataset Class

In [2]:
# VOC2012 Dataset Class
class VOCSegmentation(Dataset):
    def __init__(self, root, split='train', transform=None, target_transform=None):
        """
        Args:
            root: /kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val
            split: 'train', 'val', or 'trainval'
            transform: Transform for input images
            target_transform: Transform for segmentation masks
        """
        self.root = root
        self.split = split
        self.transform = transform
        self.target_transform = target_transform
        
        # Read image list
        split_file = os.path.join(root, 'ImageSets', 'Segmentation', f'{split}.txt')
        with open(split_file, 'r') as f:
            self.images = [x.strip() for x in f.readlines()]
        
        self.img_dir = os.path.join(root, 'JPEGImages')
        self.mask_dir = os.path.join(root, 'SegmentationClass')
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        
        # Load image
        img_path = os.path.join(self.img_dir, f'{img_name}.jpg')
        image = Image.open(img_path).convert('RGB')
        
        # Load mask
        mask_path = os.path.join(self.mask_dir, f'{img_name}.png')
        mask = Image.open(mask_path)
        
        if self.transform:
            image = self.transform(image)
        
        if self.target_transform:
            mask = self.target_transform(mask)
        else:
            mask = torch.from_numpy(np.array(mask)).long()
        
        return image, mask

In [3]:
ROOT = "/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val"

train_ds = VOCSegmentation(root=ROOT, split="train")
val_ds = VOCSegmentation(root=ROOT, split="val")
trainval_ds = VOCSegmentation(root=ROOT, split="trainval")

print("Train:", len(train_ds))
print("Val:", len(val_ds))
print("Train+Val:", len(trainval_ds))


'/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val'

## VGGNet with intermediate outputs

In [4]:
class VGGNet(VGG):
    def __init__(self, pretrained=True, model='vgg16', requires_grad=True, remove_fc=True):
        super().__init__(make_layers(cfg[model]))
        self.ranges = ranges[model]

        if pretrained:
            exec("self.load_state_dict(models.%s(pretrained=True).state_dict())" % model)

        if not requires_grad:
            for param in super().parameters():
                param.requires_grad = False

        if remove_fc:
            del self.classifier

    def forward(self, x):
        output = {}
        
        # Get output after each maxpool layer
        for idx in range(len(self.ranges)):
            for layer in range(self.ranges[idx][0], self.ranges[idx][1]):
                x = self.features[layer](x)
            output["x%d" % (idx + 1)] = x

        return output

In [5]:
# different version of vggnet
ranges = {
    'vgg11': ((0, 3), (3, 6),  (6, 11),  (11, 16), (16, 21)),
    'vgg13': ((0, 5), (5, 10), (10, 15), (15, 20), (20, 25)),
    'vgg16': ((0, 5), (5, 10), (10, 17), (17, 24), (24, 31)),
    'vgg19': ((0, 5), (5, 10), (10, 19), (19, 28), (28, 37))
}

cfg = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


In [6]:
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

## FCN Architecture

In [7]:
# FCN32s - No skip connections
class FCN32s(nn.Module):
    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu = nn.ReLU(inplace=True)
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1 = nn.BatchNorm2d(512)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn5 = nn.BatchNorm2d(32)
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        output = self.pretrained_net(x)
        x5 = output['x5']  # size=(N, 512, x.H/32, x.W/32)

        score = self.bn1(self.relu(self.deconv1(x5)))     # size=(N, 512, x.H/16, x.W/16)
        score = self.bn2(self.relu(self.deconv2(score)))  # size=(N, 256, x.H/8, x.W/8)
        score = self.bn3(self.relu(self.deconv3(score)))  # size=(N, 128, x.H/4, x.W/4)
        score = self.bn4(self.relu(self.deconv4(score)))  # size=(N, 64, x.H/2, x.W/2)
        score = self.bn5(self.relu(self.deconv5(score)))  # size=(N, 32, x.H, x.W)
        score = self.classifier(score)                    # size=(N, n_class, x.H/1, x.W/1)

        return score


In [11]:
# FCN16s - Skip connection from pool4
class FCN16s(nn.Module):
    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu = nn.ReLU(inplace=True)
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1 = nn.BatchNorm2d(512)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn5 = nn.BatchNorm2d(32)
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        output = self.pretrained_net(x)
        x5 = output['x5']  # size=(N, 512, x.H/32, x.W/32)
        x4 = output['x4']  # size=(N, 512, x.H/16, x.W/16)

        score = self.relu(self.deconv1(x5))               # size=(N, 512, x.H/16, x.W/16)
        score = self.bn1(score + x4)                      # element-wise add, size=(N, 512, x.H/16, x.W/16)
        score = self.bn2(self.relu(self.deconv2(score)))  # size=(N, 256, x.H/8, x.W/8)
        score = self.bn3(self.relu(self.deconv3(score)))  # size=(N, 128, x.H/4, x.W/4)
        score = self.bn4(self.relu(self.deconv4(score)))  # size=(N, 64, x.H/2, x.W/2)
        score = self.bn5(self.relu(self.deconv5(score)))  # size=(N, 32, x.H, x.W)
        score = self.classifier(score)                    # size=(N, n_class, x.H/1, x.W/1)

        return score


In [12]:
# FCN8s - Skip connections from pool4 and pool3
class FCN8s(nn.Module):
    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu = nn.ReLU(inplace=True)
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1 = nn.BatchNorm2d(512)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn5 = nn.BatchNorm2d(32)
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        output = self.pretrained_net(x)
        x5 = output['x5']  # size=(N, 512, x.H/32, x.W/32)
        x4 = output['x4']  # size=(N, 512, x.H/16, x.W/16)
        x3 = output['x3']  # size=(N, 256, x.H/8,  x.W/8)

        score = self.relu(self.deconv1(x5))               # size=(N, 512, x.H/16, x.W/16)
        score = self.bn1(score + x4)                      # element-wise add, size=(N, 512, x.H/16, x.W/16)
        score = self.relu(self.deconv2(score))            # size=(N, 256, x.H/8, x.W/8)
        score = self.bn2(score + x3)                      # element-wise add, size=(N, 256, x.H/8, x.W/8)
        score = self.bn3(self.relu(self.deconv3(score)))  # size=(N, 128, x.H/4, x.W/4)
        score = self.bn4(self.relu(self.deconv4(score)))  # size=(N, 64, x.H/2, x.W/2)
        score = self.bn5(self.relu(self.deconv5(score)))  # size=(N, 32, x.H, x.W)
        score = self.classifier(score)                    # size=(N, n_class, x.H/1, x.W/1)

        return score

In [13]:
# FCNs - Skip connections from all pools
class FCNs(nn.Module):
    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu = nn.ReLU(inplace=True)
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1 = nn.BatchNorm2d(512)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn5 = nn.BatchNorm2d(32)
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        output = self.pretrained_net(x)
        x5 = output['x5']  # size=(N, 512, x.H/32, x.W/32)
        x4 = output['x4']  # size=(N, 512, x.H/16, x.W/16)
        x3 = output['x3']  # size=(N, 256, x.H/8,  x.W/8)
        x2 = output['x2']  # size=(N, 128, x.H/4,  x.W/4)
        x1 = output['x1']  # size=(N, 64, x.H/2,  x.W/2)

        score = self.bn1(self.relu(self.deconv1(x5)))     # size=(N, 512, x.H/16, x.W/16)
        score = score + x4                                # element-wise add
        score = self.bn2(self.relu(self.deconv2(score)))  # size=(N, 256, x.H/8, x.W/8)
        score = score + x3                                # element-wise add
        score = self.bn3(self.relu(self.deconv3(score)))  # size=(N, 128, x.H/4, x.W/4)
        score = score + x2                                # element-wise add
        score = self.bn4(self.relu(self.deconv4(score)))  # size=(N, 64, x.H/2, x.W/2)
        score = score + x1                                # element-wise add
        score = self.bn5(self.relu(self.deconv5(score)))  # size=(N, 32, x.H, x.W)
        score = self.classifier(score)                    # size=(N, n_class, x.H/1, x.W/1)

        return score


## Transforms dataset

In [14]:
from torchvision import transforms

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),          # FCN expects fixed size
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [15]:
mask_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=Image.NEAREST),
    transforms.Lambda(lambda x: torch.from_numpy(np.array(x)).long())
])


In [16]:
VOC_ROOT = "/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val"

## Dataset and DataLoader

In [17]:
train_dataset = VOCSegmentation(
    root=VOC_ROOT,
    split='train',
    transform=image_transform,
    target_transform=mask_transform
)

val_dataset = VOCSegmentation(
    root=VOC_ROOT,
    split='val',
    transform=image_transform,
    target_transform=mask_transform
)


In [18]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)


In [19]:
img, mask = train_dataset[0]
print(img.shape)     # (3, 224, 224)
print(mask.shape)    # (224, 224)
print(torch.unique(mask))


torch.Size([3, 224, 224])
torch.Size([224, 224])
tensor([  0,   1,  15, 255])


## Training Loops

In [20]:
from tqdm import tqdm

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for images, masks in tqdm(dataloader, desc='Training'):
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)            # (N, C, H, W)
        loss = criterion(outputs, masks)  # ignore_index=255 handles void

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)


In [21]:
@torch.no_grad()
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0

    for images, masks in tqdm(dataloader, desc='Validation'):
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        running_loss += loss.item()

    return running_loss / len(dataloader)


## IoU

In [22]:
def fast_hist(pred, label, n_class):
    mask = (label >= 0) & (label < n_class)
    return torch.bincount(
        n_class * label[mask] + pred[mask],
        minlength=n_class ** 2
    ).reshape(n_class, n_class)


In [23]:
@torch.no_grad()
def evaluate_iou(model, dataloader, device, num_classes):
    model.eval()
    hist = torch.zeros((num_classes, num_classes), device=device)

    for images, masks in tqdm(dataloader, desc='Evaluating IoU'):
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        valid = masks != 255
        hist += fast_hist(
            preds[valid],
            masks[valid],
            num_classes
        )

    iou = torch.diag(hist) / (
        hist.sum(1) + hist.sum(0) - torch.diag(hist) + 1e-10
    )

    mean_iou = torch.nanmean(iou).item()
    return iou.cpu().numpy(), mean_iou


In [24]:
    # Hyperparameters
    BATCH_SIZE = 4
    LEARNING_RATE = 1e-4
    NUM_EPOCHS = 50
    NUM_CLASSES = 21
    MODEL_TYPE = 'fcn8s'  # 'fcn32s', 'fcn16s', 'fcn8s', or 'fcns'

In [25]:
    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

Using device: cuda


In [26]:
# Create VGGNet backbone
vgg_model = VGGNet(pretrained=True, model='vgg16', requires_grad=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:02<00:00, 195MB/s] 


In [27]:
# Create FCN model
model_dict = {
    'fcn32s': FCN32s,
    'fcn16s': FCN16s,
    'fcn8s': FCN8s,
    'fcns': FCNs
}

model = model_dict[MODEL_TYPE](pretrained_net=vgg_model, n_class=NUM_CLASSES).to(device)
print(f'\nUsing {MODEL_TYPE.upper()} architecture')


Using FCN8S architecture


In [28]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=255)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

## Training

In [29]:
best_miou = 0.0
NUM_EPOCHS = 50

for epoch in range(NUM_EPOCHS):
    print(f'\n{"="*60}')
    print(f'Epoch {epoch+1}/{NUM_EPOCHS}')
    print(f'{"="*60}')

    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = validate_epoch(model, val_loader, criterion, device)

    print(f'Train Loss: {train_loss:.4f}')
    print(f'Val   Loss: {val_loss:.4f}')

    if (epoch + 1) % 5 == 0:
        print('\nEvaluating mIoU...')
        ious, mean_iou = evaluate_iou(model, val_loader, device, NUM_CLASSES)
        print(f'Mean IoU: {mean_iou*100:.2f}%')

        voc_classes = [
            'background', 'aeroplane', 'bicycle', 'bird', 'boat',
            'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
            'diningtable', 'dog', 'horse', 'motorbike', 'person',
            'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
        ]

        print('\nPer-class IoU:')
        for cls, iou in zip(voc_classes, ious):
            print(f'{cls:15s}: {iou*100:5.2f}%')

        if mean_iou > best_miou:
            best_miou = mean_iou
            torch.save(model.state_dict(), f'fcn_{MODEL_TYPE}_best.pth')
            print(f'✓ Model saved! Best mIoU: {best_miou*100:.2f}%')

    scheduler.step()



Epoch 1/50


Validation: 100%|██████████| 363/363 [00:11<00:00, 31.12it/s]


Train Loss: 2.4259
Val   Loss: 1.9698

Epoch 2/50


Validation: 100%|██████████| 363/363 [00:11<00:00, 30.50it/s]


Train Loss: 1.7582
Val   Loss: 1.5420

Epoch 3/50


...

Train Loss: 0.0218
Val   Loss: 0.8991

Epoch 50/50


Validation: 100%|██████████| 363/363 [00:12<00:00, 28.76it/s]


Train Loss: 0.0208
Val   Loss: 0.8420

Evaluating mIoU...


Evaluating IoU: 100%|██████████| 363/363 [00:12<00:00, 28.55it/s]


Mean IoU: 41.43%

Per-class IoU:
background     : 88.81%
aeroplane      : 62.28%
bicycle        : 29.98%
bird           : 37.50%
boat           : 33.08%
bottle         : 32.29%
bus            : 57.63%
car            : 56.13%
cat            : 51.68%
chair          : 13.28%
cow            : 28.17%
diningtable    : 24.73%
dog            : 40.29%
horse          : 29.62%
motorbike      : 48.12%
person         : 65.04%
pottedplant    : 26.27%
sheep          : 38.25%
sofa           : 17.27%
train          : 43.80%
tvmonitor      : 45.75%
✓ Model saved! Best mIoU: 41.43%
